# HiMAP tutorial: mock community data

In [ ]:
library(himap)

Specify full main path and load raw FASTQ files. If ~ is not expanded, blastn breaks.

In [ ]:
out_path = path.expand('~/cloud/research/microbiome/zheng-2015/tutorial')

In [ ]:
fastq_path = file.path(out_path, 'fastq')
fq_fwd = read_files(fastq_path, 'R1')
fq_rev = read_files(fastq_path, 'R2')

Generate output files for the entire pipeline:

In [ ]:
sample_ids = sampleids_from_filenames(fq_fwd, separator='_')
fq_mer = file.path(out_path, 'himap_merged', paste0(sample_ids, '.fastq'))
fq_tri = file.path(out_path, 'himap_trimmed', paste0(sample_ids, '.fastq'))
fq_fil = file.path(out_path, 'himap_filtered', paste0(sample_ids, '.fastq'))

## Pre-processing
Merge reads, trim PCR primers and perform quality control and final trimming.

In [ ]:
mergestats = merge_pairs(fq_fwd, fq_rev, fq_mer, verbose=T)

Primer set used for amplifying V3-V4 region was 341F and 805R, which are labelled as 'V3-V4-2' in our reference table:

In [ ]:
himap_option('blast_dbs')[6]

In [ ]:
trimstats = pcr_primer_trimmer(fq_mer, fq_tri, region='V3-V4-2', verbose=T)

In [ ]:
trimstats

In [ ]:
seqlen.ft = sequence_length_table(fq_tri)
trim_length = ftquantile(seqlen.ft, 0.01)
trim_length

In [ ]:
plot(seqlen.ft)

In [ ]:
filtstats = dada2::filterAndTrim(fq_tri, fq_fil, truncLen=trim_length, maxEE=2, multithread=T, verbose=T)

In [ ]:
dada_result = dada_denoise(fq_fil, fq_tri)
saveRDS(dada_result, file.path(out_path, 'dada_result'))

In [ ]:
ab.dt = sequence_abundance(dada_result)

In [ ]:
head(ab.dt)

In [ ]:
blast_output = blast(ab.dt, blast_output_file, region='V3-V4', word_size=50, max_target_seqs=500, verbose=T)

In [ ]:
saveRDS(blast_output, file.path(out_path, 'blast_output'))

In [ ]:
osu_ab.dt = abundance(ab.dt, blast_output)

In [ ]:
osu_ab.dt

In [ ]:
osu_ab.dt[, .N, by=sample_id]

In [ ]:
saveRDS(osu_ab.dt, file.path(out_path, 'osu_ab.dt'))

In [ ]:
library(data.table)

In [ ]:
osu_ab.dt[species %like% 'Clostridium']

In [ ]:
osu_ab.dt[osu_id==2783]

In [ ]:
blast_output$[osu_id==2783]

In [ ]:
blast_output$alignments[pctsim==100]

In [ ]:
ab.dt